In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Detect outliers using KMeans Clustering

### Generate data

In [2]:
n_samples = 50
n_outliers = int(n_samples*0.25)
n_features = 2

gen_cov = np.eye(n_features)
gen_cov[0, 0] = 2.
gen_cov[1, 1] = 1.5
X = np.dot(np.random.randn(n_samples, n_features), gen_cov)

### Add some outliers

In [3]:
outliers_cov = np.eye(n_features)
outliers_cov[0,0] = 5.
outliers_cov[1,1] = 7.
X[-n_outliers:] = np.dot(np.random.randn(n_outliers, n_features), outliers_cov)